<img src="https://github.com/christopherhuntley/BUAN6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **BUAN 6510**
# **Lesson 11: NoSQL and Performance Tradeoffs** 
_For when a centralized row store just won't work_

## **Learning Objectives**
### **Theory / Be able to explain ...**
- 

### **Skills / Know how to ...**
- 

--------
## **LESSON 11 HIGHLIGHTS**

In [ ]:
#@title Run this cell if video does not appear
%%html
<div style="max-width:1000px">
  <div style="position: relative;padding-bottom: 56.25%;height: 0;">
    <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" rel="0" modestbranding="1"  src="https://www.youtube.com/embed/uRoW2sojmsE" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
  </div>
</div>

## **BIG PICTURE: Physical Design Alternatives**
In this lesson we will explore various alternatives to the traditional relational DBMS. Some of the alternatives will drop SQL itself, which would seem like heresy. However, there are actually some important use cases where one might not want to use a SQL-based solution. Even when SQL is the right decision there are plenty of options to choose from that fit some use cases better than others. 

In Lesson 5 we discussed four basic design tradeoffs:
- Minimizing Storage Space
- Maximizing Calculation Speed
- Maximizing Coherency
- Minimizing Data Corruption Risk

These tradeoffs exist regardless of the technology used. That's why they are *design* tradeoffs. However, there is more to database system than design. Sometimes we have to broaden our scope to consider other alternatives. 

Here are four technical considerations that go beyond logical design:
- **Flexibility / Developer Experience**  
  How easy is it for developers to learn and use the technology? It may make sense to use technology that is closer to what your programmers use everyday. 
- **Scalability / Performance Speed and Cost**  
  There is a natural tradeoff between speed and cost. Technology that works at Big Data Scale makes that tradeoff explicit. How much does each GB of storage cost? How about each GB/sec of query throughput? If we are willing to wait a little longer for each query can we save some money? 
- **Consistency / Timeliness**  
  Data is worthless if it is not available when you need it. If we don't need data to be instantaneously available, how long can we wait? If we need the data right now, then how tolerant are we of anomalies and other imperfections? If we use data replication to speed up access, do we need all copies to be 100% consistent? 
- **Technical Maturity / Technical Debt**  
  Database technology is always evolving, with new solutions coming out all the time. The newest technology may score well on the above considerations but also may come with bugs and other problems that need to be worked out. Meanwhile, older technology may be rock solid but also may limit your choices going forward. There is always the risk of being stuck with obsolete technology when your competition is beating you with something newer. 

We will start with the first two considerations, developer experience and performance, which are generally used as the rationale for NoSQL technologies that don't (necessarily) adhere to the relational database model. Then we will follow up with strategies that can further improve the speed and cost performance for any technology, providing you are able to make the right consistency and maturity tradeoffs.


---
## **NoSQL Databases**
The term "NoSQL" was first coined in the late 1990s and gained popularity among application programmers about a decade later. It refers to databases that do not rely on the relational model. That does not mean that SQL (or some close approximation) isn't used, but rather, that NoSQL systems extend beyond the traditional relational model. 

For this reason some interpret NoSQL as "Not only SQL" rather than exclusively no use of SQL at all. In fact, each of the NoSQL technologies surveyed here *could* in fact be implemented in SQL, and we will try to use relational models to explain how each related to SQL and how it extends beyond it. 

Why would we even need to go beyond SQL? 
- **Developer Experience (DX):** As we have discussed before, there is a natural impedance mismatch between a declarative language like SQL and a more imperative application development language like Python, Java, or JavaScript. NoSQL technologies remove much of the discomfort some programmers feel when using SQL.
- **Performance:** Selectively relaxing the rules of the relational model can sometimes bring speed and cost benefits that outweigh the integrity protections of the relational model. 

For each of the models below we will discuss how it differs from the standard relational model, potential DX or performance benefits, and the most common usage scenarios. 

### **Key-Value Stores**
Key-Value (KV) stores are most useful when a table is very sparse, like this section of the NBA PlayLog data set. 

![Sparse Table](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Sparse_Table.png)

If most values in a table are blank, then why waste time and effort recording them in rows and columns. Instead, just store the data you actually have, tagged to suit however you may need to retrieve it. 

In relational terms, a KV store is just a single two column table 
`kv_store(`**`key`**`, value)`

where
- **`key`** is a unique index
- `value` is a data to be stored

Usually the data type of the `value` is either encoded data itself (e.g., like i2'15' or s5'Steve`), encoded in the key, or assumed to be text. 

> **heads Up:** We have already seen an example in Lesson 5. The Entity-Attribute-Value model can, with the right modeling conventions, be seen as a kind of KV store. The key would be a composite of the entity and attribute. 

In case you are wondering how we can replace a two-dimensional table with rows and columns with a one dimensional KV store. We do it exactly like we would with composite indexes in the relational model, with the row and column encoded in the key. It's all in the patterns we use when constructing the keys.

For example, the following could be used to record the `assist`,`away`, etc. columns as key-value pairs:

| **Key** | **Value**|
| --- | ---|
| 2:away | Derrick Favors |
| 2:home | Marc Gasol |
| 18:opponent | Kyle Lowry |
| 20:num | 1 |
| 21:num | 2 |

KV Stores first are commonly used for creating data caches for the web. Here, for example, is the data that Colab is keeping about *this page* while I have it open in my web browser: 
![](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Colab_Local_Storage.png)

Similar caching technology is used by webservers to minimize reads from the disk storage. If the same CSS is used on every page then why read it from disk every time? For the server will instead use a highly optimized KV store. 

#### **Wide-Column KV Stores**
Wide-Column data stores extend the KV model to allow data storage as multiple columns:
- The keys are just like an other KV store.
- The columns are encoded like a SQL `STRUCT` (or JSON object), with each column having a name and a value. 

The columns for each key may vary, like this:

| **Key** | **Value**|
| --- | ---|
| 2 | {away:Derrick Favors, home: Marc Gasol} |
| 18 | {opponent:Kyle Lowry} |
| 20 | {num:1} |
| 21 | {num:2} |

The effect is to condense the aleady very space-efficient key-value model by eliminating overlapping keys. In this case we eliminated a row of data storage by sharing the key `2` for the `away` and the `home` columns. 

#### **Summary**

- Pros
  - very fast storage and retrieval (when applicable)
  - compact storage 
  - programmer-friendly
- Cons
  - schema by convention instead of rules
  - potential for schema drift as new key types proliferate
  - no FKs or analogous way to integrate data across keys
- When to use
  - with sparse or highly volatile data where the keys need to be flexible
  - as local data stores in application development
  - when caching data to speed up application performance
- Example Products
  - [Varnish](https://varnish-cache.org/)
  - Nginx
  - Squid
  - Memcached
  - redis
  - AWS DynamoDB

### **Document Stores**
Document stores build on the KV model to construct arbitrarily complex (and data rich) databases of *semi-structured* data. Semi-structured data has a schema (so we can interpret and process it), just like any other data model, but not one we can know about in advance. Specifically, in a document store each document (roughly equivalent to a table row) can have its own schema, structuring the data however it likes. We can then only know the document schema *on read*, whereas for the relational model it is known *on write*. 

The classic example of a document storage format is that used by Microsoft Word, which acts a *container* for components (blocks of text, titles, images, tables, etc.). One can insert just about anything inside an MS Word file (even malware, unfortunately). MS Word will then compose and render the components as documents in real time so end users can read and edit the data contained inside. 

A more relevant example is JSON, which has become a ubquitious format for transmitting data over the web. Like with an MS Word document, a JSON object or list acts as a container, into which we can insert ... JSON objects and lists. The results is a tree structure, with objects and lists nested inside each other to some unspecified depth. 

Here, for example, is what a Colab notebook (this one, in fact) looks like when you open it in a text editor:
![Colab as JSON](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Colab_As_JSON.png)

Yes, the `ipynb` file format is really just highly structured JSON. Here's the same JSON in a nicer, pretty printed format:
![Prettified JSON](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Pretty_JSON.png)

A Colab Notebook is mostly a list of "cells" $-$ look for it in the screenshot $-$, where each cell has a `cell_type`, `metadata`, and `source`. The `source` is wherever we typed into the cell. 

We will come back to JSON in the **Pro Tips** section later in this lesson. 

#### **Summary**
- Pros
  - little or no impedance mismatch for programmers, especially when using JavaScript
  - very compact, especially for semi-structured data
- Cons
  - same as KV stores; complex queries are especially difficult
  - schema on read complicates app design and development; potentially buggy
- When to use
  - for local storage or web transmission of data
  - for complex hierarchically-structured data, where documents are composed of nested components
  - When storing "objects" in relational databases
- Example Products
  - CouchDB
  - mongoDB
  - AWS DynamoDB
  - Google Cloud Firestore

### **Graph Databases**
A graph database organizes data into three kinds of structures:
- A set of **nodes** that represent entities within the domain
- A set of **edges** (or arcs) connecting nodes to represent relationships
- **Properties** that represent the attributes of each entity

SHOW AN EXAMPLE FROM THE MOVIES TONIGHT DATASET

| ![Neo4J Screenshot](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Neo4J_Screenshot_Annotated.png) |
|:---:|
| Original Image Source: [*Graph Databases, Linked Data, RDF, and the Semantic Web Wasteland*]( https://medium.com/@eisenzopf/graph-databases-linked-data-rdf-and-the-semantic-web-wasteland-69e9f4347a5b) |


On a superficial level graph databases sound a lot like relational databases, with nodes = entities, properies = attributes, and edges = relationships.  A graph database, however, is much more flexible in how it defines these things: 
- Since we are not using relations, there is no need to separate the entities by type. If we like, **we can treat each node as a unique type, with its own distinct properties (kept as Key-Value pairs).** 
- Whereas a relational database treats relationships as being between entity types, graph databases define them on specific node pairs. **Any pair of nodes can be connected by an edge, regardless of type.** 
- While each node has a unique identifier, there is no concept of type-based foreign key columns and constraints. Instead, **each edge is treated like an entity with a unique identifier.** The edge then has two implicit properties to reference the nodes it connects. 

With all that said, a graph database *can* be mapped into a very specific relational model where:
- There are three types of entities: nodes, edges, and properties.
- Edge entities are directional, with a start node and an end node.
- Properties are associated with nodes or edges as needed.

The ERD might look something like this:

So if the graph database is really just a special case of the relational model, then what's the point? For some applications a graph database can be much, much faster than a relational database. For example, consider the database behind the [Six Degrees of Kevin Bacon](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon) game, where each working actor has a "Bacon Number" BN that represents the number of edges needed to connect them to Kevin Bacon. Kevin Bacon's BN is 0. Anyone who has appeared in a movie with him has a BN of 1. Anyone who has appeared with anyone whose BN=1 has a BN of 2, etc. The game is to guess an obscure actor's BN, with super bonus points for finding a movie actor *anywhere in the world* without a BN. 

Let all the movie credits in the world be stored in a single table called `credits`, with one row per actor and movie. To keep things as simple as possible we will assume that 
- Each actor and each movie has a unique identifier (`actor_id` and `movie_id`).
- We already know the `actor_id` identifiers for Kevin Bacon and the other actor in question.

Then to find all people with BN=1 we would join the credits table with itself one time:

```sql
SELECT distinct c1.actor_id 
FROM credits AS c0 
    JOIN credits AS c1 USING (movie_id)
WHERE c0.actor_id = kevin_bacon_id
```
(If we want to look for a particular actor, we can just include them as `c1.actor_id` in the `WHERE` clause. If there are no results then the actor's BN is greater than 1.) 

To find all the movies these BN=1 actors appeared in would do another join:
```sql
SELECT distinct c2.movie_id 
FROM credits AS c0 
    JOIN credits AS c1 ON (c0.movie_id = c1.movie_id)
    JOIN credits AS c2 ON (c1.actor_id = c2.actor_id)
WHERE c0.actor_id = kevin_bacon_id
```

We can then repeat the process with another join to get the BN=2 actors. To get the BN=3 actors we would add on two more joins to the chain. If we repeat the process out to BN=6 (the theoretical maximum for a working movie actor) then we would have 11 joins in the chain. 

```sql
SELECT distinct c11.actor_id 
FROM credits AS c0 
    JOIN credits AS c1 ON (c0.movie_id = c1.movie_id) # BN=1

    JOIN credits AS c2 ON (c1.actor_id = c2.actor_id)
    JOIN credits AS c3 ON (c2.movie_id = c3.movie_id) # BN=2

    JOIN credits AS c4 ON (c3.actor_id = c4.actor_id)
    JOIN credits AS c5 ON (c4.movie_id = c5.movie_id) # BN=3

    JOIN credits AS c6 ON (c5.actor_id = c6.actor_id)
    JOIN credits AS c7 ON (c6.movie_id = c7.movie_id) # BN=4

    JOIN credits AS c8 ON (c7.actor_id = c8.actor_id)
    JOIN credits AS c9 ON (c8.movie_id = c9.movie_id) # BN=5

    JOIN credits AS c10 ON (c9.actor_id = c10.actor_id)
    JOIN credits AS c11 ON (c10.movie_id = c11.movie_id) # BN=6
WHERE c0.actor_id = kevin_bacon_id
```

That seems ugly but still plenty doable, until you realize that **the `credits` table might have 50 million rows.** There are about 10 million credited movie actors, with an average of about 5 movies per career. **Even with top hardware the query could take virtually forever. For a graph database, however, the specific path of edges connecting any actor to any other in the network can be found in less than a second.**

The secret is in the specificity of the graph database model. We don't need to explore all possible actor-to-movie-to-actor connections. Instead we only need to follow the ones along the connecting path, which can be found fairly quickly via dynamic programming. 

The achilles heel of the graph database model is bulk computation. If we need to do the same repetitive operation millions of times, perhaps concurrently, then a relational or perhaps a document database is likely a better fit, depending on the operation in question. Graph database are designed for searching and representation, and they do that very well, but not so much for anything else. 

#### **Summary**
- Pros
  - flexible data model, with nodes, entities, and properties as first-class entity types
  - scales well for associative datasets with lots of relationships
- Cons
  - same as KV stores; complex queries are especially difficult 
  - while searchable, lacks most of the computational power of the relational model
  - treating each edges and property as a separate entity may not be space efficient
- When to use
  - whenever the data is naturally represented as nodes, edges, and properties
  - for network-focused apps like customer relationship management, road navigation, certain search engine operations, and file systems
  - for semantic maps (e.g., in text analytics) where the relationships represent interactions, forces, hierarchies, etc.
- Example Products
  - Neo4j
  - RedisGraph
  - AWS Neptune

  






---
## **Technology for Capacity and Performance**
### **Distributed Databases**
CAP
### **Row-stores vs Column-stores**

---
## **PRO TIPS: How to Work with JSON Data in SQL**






---
## **SQL AND BEYOND: Git as a Distributed DBMS**

---
## **Congratulations! You've made it to the end of Lesson 11.**

There is no Lesson 12. So maybe celebrate with your beverage of choice.  



## **On your way out ... Be sure to save your work**.
In Google Drive, drag this notebook file into your `BUAN6510` folder so you can find it next time.